In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

from numba import njit

import copy
import numpy as np
import scipy as scy

In [181]:
from numpy import nan


class Box():
    """Box-class: defining a rectangular box-shape, in which Particles can roam. One corner is always (0,0)\n
    box         : 2-dimensional array of the two lengths of the x- and y-axis, corresponding to the boundaries of the box.\n
    particles   : a list of all particles in the box\n
    n_particles : the number of particles inside of the box
    """
    def __init__(self, box_size, n_Particles:int):
        """Initializing the Box-class\n
        box_size    : a 2-dimensional array of the two lengths of the x- and y-axis, corresponding to the boundaries of the box.\n
        n_Particles : an integer of the number of Particles in the box. Needed to initialize all arrays correctly
        """
        # constants
        self.c_6        = 6.2647225     # kg/mol  *  nm**8/ns**2
        self.c_12       = 9.847044e-3   # kg/mol  *  nm**14/ns**2
        self.kB         = 1.380e-23     # J/K
        self.avogadro   = 6.022e23      # 1/mol

        self.box_size       = box_size
        # Particles statistics are no longer stored in a separate class but in arrays, which allows for easier calculations
        self.particles_pos  = np.zeros((n_Particles,2))
        self.particles_vel  = np.zeros((n_Particles,2))
        self.particles_acc  = np.zeros((n_Particles,2))

        self.particles_r    = np.zeros(n_Particles)
        self.combined_radii = self.particles_r[:,np.newaxis] + self.particles_r[np.newaxis,:]
        self.particles_m    = np.zeros(n_Particles, dtype=float)
        
        # We use distance matrices for true distances (distances_mat) in x and y coords, 
        # distances with ghost particles for cross boundary calculations (distance_ghost) in x and y coords
        # and the norm of the ghost_distances as a single float
        self.distance_mat   = np.zeros((n_Particles,n_Particles,2))
        self.distance_ghost = np.zeros((n_Particles,n_Particles,2))
        self.distance_abs   = np.zeros((n_Particles,n_Particles))
        self.distance_abs_ghost   = np.zeros((n_Particles,n_Particles))

        self.velocity_mat   = np.zeros((n_Particles,n_Particles,2))

        self.impact_mat     = np.zeros((n_Particles,n_Particles,2))

        # A vectorial force matrix to store the forces between each pair of particles in both directions
        self.force_mat      = np.zeros((n_Particles,n_Particles,2))
        # Storing the energy types currently in the system
        self.potEnergy_mat  = np.zeros((n_Particles,n_Particles,2))
        self.kinEnergy_mat  = np.zeros((n_Particles,n_Particles))

        self.temp = 0 # K

        self.n_particles = n_Particles

    def __repr__(self):
        """printing for debugging"""
        return str("This is a box of size %0.2f by %0.2f" % (self.box_size[0],self.box_size[1]) + ", with %0.2f" % (self.n_particles) + " particles")

    def random_positions(self, axis, n_particles = 0) -> None:
        """return random positions for a number of particles (only one axis)\n
        axis        : either 0 or 1.     0 = x-axis, 1 = y-axis\n
        n_particles : the number of particles for which positions should be given; default self.n_particles\n
        returns     : array of random positions
        """
        if n_particles == 0:
            n_particles = self.n_particles
        rnd = np.random.rand(n_particles)*(self.box_size[axis]-0.5)
        return rnd

    def fixedtemp_velocities(self, target_temp):
        """Calculate initial velocities, so that the initial kinetic energy corresponds to a target temperature"""
        kin_energy = self.kB * target_temp
        velocities = np.sqrt(2 * (kin_energy/self.n_particles) / (self.particles_m / self.avogadro))
        return velocities

    def fill_particles(self, radius, mass, vel, angle = [], x = [], y = [], align = 'random', grid = np.zeros(2), target_temp = 300) -> None:
        """fills the particles-array with particles\n
        radius      : The radius of particles; either as array of length n for individual radii or int/float for a general radius\n
        vel         : The absolute velocity; either as array of length n for individual velocities or int/float for a uniform initial velocity\n
        angle       : The initial angles of the particles as array of length n for individual angles; default is uniformly distributed\n
        x,y         : initial positions as array of length n; default random positions 0.5 away from border; Only used when align = 'defined'\n
        align       : Type of Particle placement: random = random positions; grid = grid-like arrangement(needs corresponding grid argument); defined = as given by x,y input\n
        grid        : 2x2 array that defines how many Particles should be in the grid on the x- and y- axis. Take care of correct number of particles!
        """
        # filling radius and mass, if given as a number for all particles
        if type(radius) == int or type(radius) == float:
            self.particles_r = np.ones(self.n_particles)*radius
        else:
            self.particles_r = radius
        self.combined_radii = self.particles_r[:,np.newaxis] + self.particles_r[np.newaxis,:]
        if type(mass) == int or type(mass) == float:
            self.particles_m = np.ones(self.n_particles)*mass
        else:
            self.particles_m = mass
            
        # randomize the angles if not given
        if len(angle) == 0:
            angle = np.random.uniform(0,2 * np.pi, self.n_particles)
        # fill velocities if given as a number for all particles
        if type(vel) == int or type(vel) == float:
            vel = np.ones(self.n_particles)*vel
        elif type(vel) == str:
            vel = self.fixedtemp_velocities(target_temp)
        
        # calculate the coressponding velocities
        self.particles_vel[:,0] = np.sin(angle) * vel
        self.particles_vel[:,1] = np.cos(angle) * vel

        # check which particle arrangement should be choosen
        if align == 'random':
            self.particles_pos[:,0] = self.random_positions(0,self.n_particles)
            self.particles_pos[:,1] = self.random_positions(1,self.n_particles)

        elif align == 'grid':
            if np.prod(grid) == self.n_particles:
                # This monster is mainly the meshgrid of coordinates, adjusted to be the correct shape.
                # Additionally the entire grid is moved by 0.1 in x and y direction to avoid Particles directly on the edge
                # It is not an error, even if vsCode thinks so
                max_rad = np.max(self.particles_r)+0.01
                self.particles_pos[:,] = np.column_stack(np.array(np.meshgrid(np.linspace(0,self.box_size[0]-max_rad,num=grid[0],endpoint=False),
                                                                              np.linspace(0,self.box_size[1]-max_rad,num=grid[1],endpoint=False),
                                                                              indexing='ij')).reshape(2,self.n_particles)) + max_rad
            else:
                print('ERROR: Grid size does not match number of particles!')
            # self.forces = np.ones((self.n_particles, self.n_particles, 2))-1
        elif align == 'left-grid':
            if np.prod(grid) == self.n_particles:
                # This monster is mainly the meshgrid of coordinates, adjusted to be the correct shape.
                # Additionally, the entire grid is moved by 0.1 in x and y direction to avoid Particles directly on the edge
                # It is not an error, even if vsCode thinks so
                max_rad = np.max(self.particles_r)+0.01
                self.particles_pos[:,] = np.column_stack(np.array(np.meshgrid(np.linspace(0,self.box_size[0]/2-max_rad,num=grid[0],endpoint=False),
                                                                              np.linspace(0,self.box_size[1]-max_rad,num=grid[1],endpoint=False),
                                                                              indexing='ij')).reshape(2,self.n_particles)) + max_rad
            else:
                print('ERROR: Grid size does not match number of particles!')
            # self.forces = np.ones((self.n_particles, self.n_particles, 2))-1
        elif align == 'defined':
            self.particles_pos[:,0] = x
            self.particles_pos[:,1] = y
        self.calculate_temp()



    def move(self, dt = 1., vel = []) -> None:
        """moving the particle in the direction, where the velocity-vector points.\n
        dt  : the time-step moving forward; default = 1\n
        vel : a velocity vector for moving in that direction during a time-step of one; default = self.vel
        """
        if len(vel) == 0:
            vel = self.particles_vel
        self.particles_pos += vel*dt
    
    def wrap_around(self) -> None:
        """For continuous borders, i.e. a particle that exits to the right is entering from the left and vice versa\n
        particles   : Particles, which should be wrapped; default self.particles\n
        returns     : array of particles with new positions
        """
        self.particles_pos = self.particles_pos % self.box_size




    def calculate_distance_matrix(self) -> None:
        """Calculates a matrix, containing x and y distances of all particles to all other particles in both directions.\n
        It is by nature not symmetric, but rather the upper triangle is negated and flipped"""
        self.distance_mat = self.particles_pos[np.newaxis, :, :] - self.particles_pos[:, np.newaxis, :] # shape: (n,n,2)

        # the ghost matrix is calculated to account for interactions that go over the boundary of the box.
        # This is adjusted, if the x or y distances is greater than half the box size, which makes the distance across boundaries shorter
        self.distance_ghost = np.where(self.distance_mat[:, :] > (0.5 * self.box_size), 
                                       self.distance_mat[:, :] - self.box_size, self.distance_mat[:, :]) # shape: (n,n,2)
        self.distance_ghost = np.where(self.distance_ghost[:, :] < -(0.5 * self.box_size), 
                                       self.distance_ghost[:, :] + self.box_size, self.distance_ghost[:, :]) # shape: (n,n,2)
        # The norm is calculated as well as the absolute distance between each Particle pair. Symmetric by nature
        self.distance_abs = np.linalg.vector_norm(self.distance_mat, axis = 2) # shape: (n,n)
        self.distance_abs_ghost = np.linalg.vector_norm(self.distance_ghost, axis = 2) # shape: (n,n)
        #self.distance_abs = np.where(self.distance_abs == 0, np.inf, self.distance_abs)
    
    def calculate_velocity_matrix(self) -> None:
        self.velocity_mat   = self.particles_vel[np.newaxis, :, :] - self.particles_vel[:, np.newaxis, :] # shape: (n,n,2)
        self.velocity_abs   = np.linalg.vector_norm(self.velocity_mat, axis = 2) # shape: (n,n)

    def calculate_force_matrix(self) -> None:
        """Calculate the forces acting from each Particle onto each particle as a directional vector stored in a matrix"""
        distance_abs_newaxis = self.distance_abs[:, :, np.newaxis] # shape: (n,n,1); used for better matrix multiplication with numpy
        # We have to check for any distances of zero between Particles, which would not allow for any force-calculation
        # We set those forces to zero
        self.force_mat = np.where(distance_abs_newaxis != 0, (12*self.c_12/distance_abs_newaxis**14-6*
                                                              self.c_6/distance_abs_newaxis**8)*self.distance_ghost, 0)

    def calculate_potEnergy_matrix(self) -> None:
        """Calculate the potential energy of the system"""
        # Again we need to account for any division by zero
        self.potEnergy_mat = np.where(self.distance_abs != 0, (self.c_12/(self.distance_abs**12)-
                                                               self.c_6/(self.distance_abs**6)), 0) # shape: (n,n)

    def calculate_kinEnergy_matrix(self) -> None:
        """Calculate the current kinetic energy in the system."""
        self.kinEnergy_mat = 0.5 * self.particles_m * np.square(np.linalg.norm(self.particles_vel, axis=1)) # shape: (n) #could be optimized maybe? square of squareroot of squares






    def calculate_temp(self):
        """Updates the temperature of the system. For this a recalculation of the kinetic energies is necessary"""
        self.calculate_kinEnergy_matrix()
        self.temp = np.sum(self.kinEnergy_mat)/self.kB/self.avogadro






    def calculate_impact_matrix(self):
        self.calculate_distance_matrix()
        self.calculate_velocity_matrix()

        v_x = self.velocity_mat[:,:,0]
        v_y = self.velocity_mat[:,:,1]

        r_x = self.distance_mat[:,:,0]
        r_y = self.distance_mat[:,:,1]

        #c
        sqrt = 2*np.sqrt((r_x*v_x+r_y*v_y)**2-(v_x**2+v_y**2)*(r_x**2+r_y**2-self.combined_radii**2))
        sqrt = np.where(sqrt==nan,np.inf, sqrt)
        
        delta_t = (-2*(r_x*v_x+r_y*v_y) + sqrt)/(2*(v_x**2+v_y**2))
        delta_t_2 = (-2*(r_x*v_x+r_y*v_y) - sqrt)/(2*(v_x**2+v_y**2))

        final_delta_t = np.where((delta_t < delta_t_2 and delta_t > 0), delta_t, delta_t_2)

        a = (self.box_size[np.newaxis,:]-self.particles_pos)/self.particles_vel
        b = (np.zeros((self.n_particles,2))-self.particles_pos)/self.particles_vel

        t_positive = np.min(np.where(a<0,np.inf,a), axis = 1)
        t_negative = np.min(np.where(b<0,np.inf,b), axis = 1)

        t_diag = np.where(t_positive > 0, t_positive, t_negative)

        for i in range(self.n_particles):
            final_delta_t[i,i] = t_diag[i]
        self.impact_mat = final_delta_t
    
    '''def recalculate_impact_matrix(self, indices):
        if indices[0] == indices[1]:
            boundary_collide()
        else:
            for i in indices:'''



    def do_step(self):
        collision = np.divmod(self.impact_mat.argmin(), self.impact_mat.shape[1])
        time_step = float(self.impact_mat[collision])
        self.move(dt=time_step)
        self.impact_mat -= time_step
        print(collision)



In [182]:
grid=np.array([3,3])
box = Box(np.array([5,5]),int(np.prod(grid)))

In [183]:
angles = [np.pi/4,-np.pi/4]
x = [1,5]
y = [1.5,1.5]

In [184]:
#box.fill_particles(0.5,18,150)
#box.fill_particles(0.5,18,150,angles,x,y,align='defined')
box.fill_particles(0.5,0.018,150,align='grid',grid=grid)
#box.fill_particles(0.5,0.018,'blah',align='grid',grid=grid)

In [185]:
box.calculate_impact_matrix()

/tmp/ipykernel_733/369089068.py:226: RuntimeWarning: invalid value encountered in sqrt
  sqrt = 2*np.sqrt((r_x*v_x+r_y*v_y)**2-(v_x**2+v_y**2)*(r_x**2+r_y**2-self.combined_radii**2))
/tmp/ipykernel_733/369089068.py:229: RuntimeWarning: invalid value encountered in divide
  delta_t = (-2*(r_x*v_x+r_y*v_y) + sqrt)/(2*(v_x**2+v_y**2))
/tmp/ipykernel_733/369089068.py:230: RuntimeWarning: invalid value encountered in divide
  delta_t_2 = (-2*(r_x*v_x+r_y*v_y) - sqrt)/(2*(v_x**2+v_y**2))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [179]:
box.impact_mat

array([[ 0.03016196,  0.01775881,         nan,         nan,         nan,
                nan,         nan,  0.00851454,         nan],
       [ 0.01775881,  0.02999405,         nan,  0.00599924,  0.00167262,
         0.00484496,         nan,  0.00716737,  0.00883458],
       [        nan,         nan,         inf,         nan,  0.00807796,
        -0.08302503,         nan,         nan,         nan],
       [        nan,  0.00599924,         nan,  0.03091011,         nan,
         0.00870231,         nan,  0.00417116,  0.00857041],
       [        nan,  0.00167262,  0.00807796,         nan,  0.12759362,
         0.0033655 , -0.0149616 ,         nan,         nan],
       [        nan,  0.00484496, -0.08302503,  0.00870231,  0.0033655 ,
                inf,         nan,  0.01013435,  0.01038226],
       [        nan,         nan,         nan,         nan, -0.0149616 ,
                nan,         inf,         nan,         nan],
       [ 0.00851454,  0.00716737,         nan,  0.00417116,   

In [180]:
box.do_step()

(np.int64(0), np.int64(2))
